# Database maker

**calquery** is a tool that queries the VLA Calibrator List and provides you with all the information necessary for determining what the best calibrators would be for your observation based on your requirements.

In [12]:
import pymongo
import os
from pymongo import MongoClient
client = MongoClient('localhost', 27017)


In [13]:
# Create the database
db = client['calquery_db']

# Create the collection
collection = db['calibrators']

In [29]:
# Path to the directory containing the tiny .txt files
input_dir = 'split_calibrators/'

# Function to parse individual .txt file
def parse_calibrator_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
        # Parse the first line for the main calibrator details
        first_line = lines[0].strip().split()
        calibrator = {
            "iau_name": first_line[0],
            "equinox": first_line[1],
            "pc": first_line[2],
            "ra": first_line[3],
            "dec": first_line[4],
            "pos_ref": first_line[5] if len(first_line) > 5 else None,
            "alt_name": first_line[6] if len(first_line) > 6 else None,
            "bands": []
        }

        # Skip to the lines with band data
        band_lines = lines[5:]  # Skipping 2nd to 5th rows

        for line in band_lines:
            if line.strip() and not line.startswith(('===', '---')):
                data = line.strip().split()
                try:
                    band_data = {
                        "band": data[0],
                        "band_name": data[1] if len(data) > 1 else None,
                        "config_a": data[2] if len(data) > 2 else None,
                        "config_b": data[3] if len(data) > 3 else None,
                        "config_c": data[4] if len(data) > 4 else None,
                        "config_d": data[5] if len(data) > 5 else None,
                        "flux_jy": None if data[6] == '?' else float(data[6]) if data[6] else None,
                        "uvmin_kl": None,
                        "uvmax_kl": None,
                        "visplot": None
                    }
                    # Check for uvmin_kl and uvmax_kl values
                    if len(data) > 7:
                        try:
                            band_data["uvmin_kl"] = float(data[7])
                        except ValueError:
                            band_data["visplot"] = data[7]
                    if len(data) > 8:
                        try:
                            band_data["uvmax_kl"] = float(data[8])
                        except ValueError:
                            band_data["visplot"] = data[8]
                    if len(data) > 9:
                        band_data["visplot"] = data[9]
    
                    calibrator["bands"].append(band_data)
                
                except IndexError as e:
                    print(f"IndexError in file {file_path}: {e}")
                    # Optionally, you can log the problematic line
                    print(f"Problematic line: {line.strip()}")
                    continue

        return calibrator

# Process each file in the input directory
for file_name in os.listdir(input_dir):
    if file_name.endswith('.txt'):
        file_path = os.path.join(input_dir, file_name)
        calibrator_data = parse_calibrator_file(file_path)
        collection.insert_one(calibrator_data)

print("Data inserted successfully!")


IndexError in file split_calibrators/0253+180.txt: list index out of range
Problematic line: 2cm    U                0.00
IndexError in file split_calibrators/0627-058.txt: list index out of range
Problematic line: 90cm    P  S               33           10
IndexError in file split_calibrators/2149-130.txt: list index out of range
Problematic line: 6cm    C  X X X X
IndexError in file split_calibrators/2149-130.txt: list index out of range
Problematic line: 2cm    U  X X X X
Data inserted successfully!
